<a href="https://colab.research.google.com/github/diegobezerra80/gestao-clientes-premium/blob/main/notebook/Santander_DIO_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# Extrair os IDs dos clientes elegíveis para upgrade de categoria
# e ajuste de limite de crédito
df = pd.read_csv("../data/clientes_classificados_premium.csv")
users_id = df['UserID'].to_list()

In [7]:
# Lista de clientes
df_clientes = pd.read_csv('../data/clientes.csv')
clientes = df_clientes.to_dict(orient='records')


In [8]:
# Atualizar os clientes para premium
def atualizar_cliente(cliente):
  cliente['Limite_Cartao'] = 10000
  cliente['Classificacao'] = 'Premium'
  return cliente
clientes_update = [atualizar_cliente(cliente) if cliente['id'] in users_id
                   else cliente for cliente in clientes]

In [9]:
# Adicionar news ao clientes premium e mandando uma mensagem personalizada de agradecimento
from google import genai
from google.colab import userdata

API_KEY = userdata.get("GEMINI_API_KEY")
client = genai.Client(api_key=API_KEY)

mensagens = []
for cliente in clientes_update:
  if cliente['Classificacao'] == "Premium":
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=f"Escreva uma única mensagem curta e simpática de agradecimento para o cliente premium {cliente['Nome']}. A mensagem deve ser educada, cordial e direta, sem incluir explicações adicionais.",
    )
    cliente['News'] = response.text


In [10]:
# Salvando em um arquivo csv
df_update = pd.DataFrame(clientes_update)
df_update.to_csv("../output/clientes_update.csv", index=False)